In [1]:
import pandas as pd
import numpy as np

from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns
import math

import time

import warnings
warnings.filterwarnings("ignore")

In [2]:
vic_data = pd.read_csv('../../../data/victoria.csv')

In [3]:
# Constant Variables 
TIME = 'Time (UTC+10)'
PRICE = 'Regions VIC Trading Price ($/MWh)'
GENERATION = 'Regions VIC Trading Total Intermittent Generation (MW)'
DEMAND = 'Regions VIC Operational Demand (MW)'

POWER = 300
CAPACITY = 580
EFFICIENCY = 0.9
CHARGE_EFF = 90
DISCHARGE_EFF = 90
MLF = 0.991
FIXED_OP = 8.1
VAR_OP = 0

In [4]:
def create_df(ori_df):
    """ Returns a proper dataframe with columns needed """

    df = ori_df[[TIME, PRICE]]
    df['raw_power'] = 0
    df['dispatch'] = 0
    df['revenue'] = 0
    df['opening'] = 0
    df['closing'] = 0
    
    # I removed the first row because first row of vic is 00:00:00, 
    # which is the last period from the previous year
    first_row = df.loc[0]
    df = df.drop([0], axis=0) 
    
    return df, first_row

In [5]:
def get_opencap(i, df):
    """Get the opening battery capacity for every 30-minutes interval 
    Input:
        i : Current row in df
        df : DataFrame with 'opening' and 'closing' column
    Return 
        opening_cap : Opening battery capacity. Integer dtype
    
    """

    if i != 1: #Not the first row
        df.at[i,"opening"] = df.at[i-1,"closing"]
    opening_cap = math.ceil(df.at[i, "opening"])
    
    return opening_cap
    
def get_dispatch(rawPower):
    """ Get the power dispatched for every 30-minutes interval 
    Input:
        rawPower : Current rawPower. Integer Datatype
    Return:
        dispatch : Raw_power dispatched from the market. Integer dtype
    """
    if rawPower < 0:
        eff = 1
    else:
        eff = DISCHARGE_EFF / 100
            
    dispatch = round(rawPower / 2 * eff, 0)
    return dispatch

def get_closecap(opening_cap, dispatch):
    """Get the battery closing capacity for every 30-minutes interval
    Input:
        opening_cap : Opening battery capacity. Integer dtype
        dispatch : Raw_power dispatched from the market. Integer dtype
    Return:
        closecap : Closing battery capacity. Integer dtype
    """
    if dispatch < 0:
        eff = CHARGE_EFF / 100
    else:
        eff = 100 / DISCHARGE_EFF

    closecap = math.ceil(max(0, min((opening_cap - (dispatch * eff)), CAPACITY)))
    return closecap
    
def get_revenue(price, dispatch):
    """ Get the Revenue for every 30-minutes interval
    Input:
        price : Market spot price for electricity. Float dtype
        dispatch : Raw_power dispatched from the market. Integer dtype
    Return:
        revenue : current revenue. Integer dtype
    """
    if dispatch < 0:
        factor = 1/MLF
    else:
        factor = MLF 
    revenue = round(price * dispatch * factor)
    return revenue

def raw_power(i, df, opening_cap):
    """ Get the Raw Power for every 30-minutes interval
    Input:
        i : Current row in df
        df : DataFrame with 'charge_forecaset' and 'discharge_forecast' column
    Return:
        opening_cap : Opening battery capacity. Integer dtype
    """
    raw_power = 0
    if (df.at[i,PRICE] < CHARGE_THRESHOLD):
        raw_power = -min(POWER, (CAPACITY-opening_cap)/EFFICIENCY*2)     # CHARGE
        
    elif (df.at[i,PRICE] > DISCHARGE_THRESHOLD):
        raw_power = min(POWER, opening_cap*2)      # DISCHARGE
            
    return raw_power

def calculate(df):
    """ Calculate the Battery Opening, Closing Capacity, Raw Power, Market dispatch 
        and Revenue for the entire df.
    Input:
        df : DataFrame with 'price, opening', 'closing', 'raw_power', 'dispatch'
            and 'revenue' column
    Return :
        df : DataFrame with 'price, opening', 'closing', 'raw_power', 'dispatch'
            and 'revenue' column
    """
    # Go through each 30-minute interval of df
    for i in list(df.index):
        # get current Spot Price
        price = df.at[i, PRICE]
        
        # update opening capacity
        opening_cap = get_opencap(i, df)

        # find raw_power
        rawPower = raw_power(i, df, opening_cap)
        df.at[i,'raw_power'] = rawPower

        # find market_dispatch 
        dispatch = get_dispatch(rawPower)
        df.at[i,"dispatch"] = dispatch

        # find closing_capacity   
        df.at[i,"closing"] = get_closecap(opening_cap, dispatch)

        #find revenue        
        df.at[i,"revenue"] = get_revenue(price, dispatch)
    
    return df

In [6]:
def show_result(df):
    """ Print the revenue related information computed from df.
    Input: 
        df : DataFrame with 'revenue' column.
    """
    print("Total revenue in the dataset:", df["revenue"].sum())
    print("Total days in the dataset:", len(df)/48)
    print("Revenue per day:", df["revenue"].sum() / (len(df)/48))
    
    return None

def run_all(ori_df):
    """ Run the entire data pipeline including initialisng (Data Processing), 
    finding the optimal charging and discarging period (Data Modelling), 
    calculating the revenue based on the optimal period mentioned above (Model Testing and Evaluation).
    
    Input:
        ori_df : DataFrame which contains spot_price for every 30-minute interval. 
    Return:
        df : DataFrame with 'price, opening', 'closing', 'raw_power', 'dispatch'
            and 'revenue' column.
    """
    # Start time
    start = time.time()
    # Initialise df
    df2, first_row = create_df(ori_df)
    
    # Calculate
    df = calculate(df2)
    
    # Add unused row back to df
    df.loc[0] = first_row
    df = df.sort_index()
    
    # Show the revenue       
    show_result(df)
    # End Time      
    end = time.time()
    print("Time Complexity for running the entire Algorithm 3: {time_taken}s".format(time_taken = end-start))
            
    return df

In [7]:
CHARGE_THRESHOLD = 100
DISCHARGE_THRESHOLD = 110
vic = run_all(vic_data)

Total revenue in the dataset: 33109325
Total days in the dataset: 1322.0208333333333
Revenue per day: 25044.48051436406
Time Complexity for running the entire Algorithm 3: 3.3658981323242188s


In [8]:
vic

,Time (UTC+10),Regions VIC Trading Price ($/MWh),raw_power,dispatch,revenue,opening,closing
0,2018-01-01 00:00:00,90.43,0,0,0,0,0
1,2018-01-01 00:30:00,92.46,-300,-150,-13995,0,135
2,2018-01-01 01:00:00,87.62,-300,-150,-13262,135,270
3,2018-01-01 01:30:00,73.08,-300,-150,-11062,270,405
4,2018-01-01 02:00:00,70.18,-300,-150,-10623,405,540
...,...,...,...,...,...,...,...
63452,2021-08-14 22:00:00,49.93,0,0,0,580,580
63453,2021-08-14 22:30:00,62.86,0,0,0,580,580
63454,2021-08-14 23:00:00,32.26,0,0,0,580,580
63455,2021-08-14 23:30:00,25.10,0,0,0,580,580
